<a href="https://colab.research.google.com/github/Sassotek/Mechanistic-Interpretability-for-Vision-Models-Optimization/blob/main/Mechanistic_Interpretability_for_Vision_Models_Optimization_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Notes

https://viso.ai/deep-learning/vision-transformer-vit/

https://www.datacamp.com/tutorial/building-a-transformer-with-py-torch

https://docs.pytorch.org/vision/main/models/vision_transformer.html

https://github.com/lucidrains/vit-pytorch

https://arxiv.org/pdf/2010.11929.pdf

https://www.youtube.com/watch?v=j3VNqtJUoz0

#Imports

In [ ]:
import math
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import cv2

#Kaggle
!pip install kagglehub
import kagglehub

#PyTorch modules
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import ModuleList
from torch.utils.data import DataLoader, SubsetRandomSampler, ConcatDataset
import torchvision
from torchvision import transforms,datasets
from torch.utils.data import Dataset

import random
import os
import pandas as pd
from tqdm.notebook import tqdm
import time
import seaborn as sns


from IPython.display import display, Markdown
from IPython.display import HTML


import sklearn
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, classification_report



## GDrive settings
from google.colab import drive
drive.mount('/content/drive')


# Setting the seed
torch.manual_seed(240700)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

print("Device",device)
if torch.cuda.is_available():
   torch.cuda.manual_seed(240700)
   torch.cuda.manual_seed_all(240700)

#ensure that all operations are deterministic on GPU if used,for reproducibility
torch.backends.cudnn.determinstic = True
torch.backends.cudnn.benchmark = False






Mounted at /content/drive
Device cuda


# GLOBALS

In [ ]:
directory_file_path="/content/drive/File_projects_CV"

RED = "\033[31m"
GREEN = "\033[32m"
BLUE = "\033[34m"
RESET = "\033[0m"
CYAN="\033[36m"
MAGENTA="\033[35m"

#DATA

In [ ]:
data_path = kagglehub.dataset_download("akash2sharma/tiny-imagenet")
print("Path to dataset files:", data_path)

Path to dataset files: /kaggle/input/tiny-imagenet


In [ ]:
#save_path_model= ................./ciao.pth
#save_path_opt= ..../ott.pth